In [ ]:
import pandas as pd

#INCAR data from bbsr
#https://www.inkar.de
#Vor allem: Anteil Fertiggestellte Wohnungen / Wohngebäude mit erneuerbarer Heizenergie, 2017-2019

#convert pandas series from German number format (1.000,5) to python number format (1000.5)
def make_column_numeric(se):
    se = se.str.replace('.','',regex=True)
    se = se.str.replace(',','.',regex=True)
    se = pd.to_numeric(se)
    return se

#read dataset
df = pd.read_csv("data/inkar_heating.csv", delimiter=';', header=[0,1])
df.columns = df.columns.map('_'.join)

#rename unnamed headers
df.rename({"Kennziffer_Unnamed: 0_level_1": "Kennziffer","Raumeinheit_Unnamed: 1_level_1":"Raumeinheit","Aggregat_Unnamed: 2_level_1":"Aggregated"}, axis=1,inplace=True)

#calculate neubauwohnungen total and neubauwohnungen renewable total
years = ["2018","2019"]
for year in years:
    #make columns numerics
    neubau_einwohner = "Neubauwohnungen je Einwohner_" + str(year)
    einwohner = 'Bevölkerung gesamt_' + str(year)
    neubauwohnungen_renewable_anteil = "Fertiggestellte Wohnungen mit erneuerbarer Heizenergie_" + str(year)
    neubauwohngebäude_renewable_anteil = "Fertiggestellte Wohngebäude mit erneuerbarer Heizenergie_" + str(year)
    
    df[neubau_einwohner] = make_column_numeric(df[neubau_einwohner]) 
    df[einwohner] = make_column_numeric(df[einwohner]) 
    df[neubauwohnungen_renewable_anteil] = make_column_numeric(df[neubauwohnungen_renewable_anteil]) / 100
    df[neubauwohngebäude_renewable_anteil] = make_column_numeric(df[neubauwohngebäude_renewable_anteil]) / 100

    #Neubauwohnungen = Neubauwohnungen je Einwohner * Einwohner 
    neubau = "neubauwohnungen_" + str(year)
    df[neubau] = df[neubau_einwohner] *  df[einwohner]

    #Neubauwohnungen renewable = Neubauwognungen * Anteil Renewalbe
    #Wohnungen vs. Wohngebäude: aus Dokumentation: "Im Gegensatz zum Indikator Fertiggestellte Wohngebäude mit erneuerbarer Heizenergie werden hier 
    #Errichtungen von Wohnungen in Mehrfamilienhäusern gegenüber Ein- und Zweifamilienhäuser stärker gewichtet." Bei Wohngebäuden wäre ein Mehrfamilienhaus
    #genausoviel wert wie ein Einfamilienhaus
    neubauwohnungen_renewable =  "neubauwohnungen_erneuerbareheiz_" + str(year)
    df[neubauwohnungen_renewable] = df[neubau] *  df[neubauwohnungen_renewable_anteil]

    neubauwohngebäude_renewable =  "neubauwohngebäude_erneuerbareheiz_" + str(year)
    df[neubauwohngebäude_renewable] = df[neubau] *  df[neubauwohngebäude_renewable_anteil]

df.to_csv("data/inkar_heating_output.csv")


In [ ]:
#https://www-genesis.destatis.de/genesis/online#astructure
#Baufertigstellungen neuer Gebäude: Deutschland, Jahre, Gebäudeart, Energieverwendung, Energieart
#von 2014 - 2021, auf Deutschlandebene

import plotly.graph_objects as go
import plotly_express as px
import numpy as np

def preprocess_dataset_germany_by_building_type(df,building_type):

    #Nur den Typ des Gebäudes auswählen
    df = df.loc[df["2_Auspraegung_Label"] == building_type]

    #Nur Zum Heizen verwendete Primärenergie
    df = df.loc[df2["3_Auspraegung_Code"] == "ENR-HZG-PRM-01"]

    df.rename({"Zeit": "year","BAU005__Neue_Gebaeude__Anzahl":"buildings","4_Auspraegung_Label":"energy"}, axis=1,inplace=True)

    #make building number numeric
    df["buildings"] = df["buildings"].replace({'-':'0',".":"0"})
    df["buildings"] = pd.to_numeric(df["buildings"])

    return df

def plot_heating_timeseries(viz,asProportion = True,useWohnungen = False):
    """
        Parameters
        ----------
        viz : df
            dataframe slice of your data, for example a specific landkreis
        asProportion : bool, optional
            if true, the visualisation will use proportion. otherwise, the total number is used. default is true
        useWohnungen : bool, optional
            if true, flats will be counted. otherwise, buildings will be counted (which maybe have multiple flats in them). default is false
    """
    

    #pivot table
    pivot = pd.pivot_table(viz, index='year',columns='energy', values='buildings',aggfunc=np.sum)
    pivot = pivot.reset_index(level=0)

    #create plot
    colors = px.colors.qualitative.Plotly
    fig = go.Figure()

    # dont add gesamtenergie to plot
    viz = viz[viz["energy"] != "Wohngebäude Insgesamt"]
    viz = viz[viz["energy"] != "Insgesamt"]

    #add trace per energy
    energies = viz.energy.unique()

    for i,energy in enumerate(energies):
        fig.add_traces(go.Scatter(x=pivot['year'], y = pivot[energy], mode = 'lines', line=dict(color=colors[i%10]),name=energy))
    
    fig.show()

def plot_heating_bars(viz,asProportion = True,useWohnungen = False):

    viz.rename({"Zeit": "year","BAU017__Wohngebaeude_insgesamt__Anzahl":"buildings","Heizenergie_Auspraegung_Label":"energy",
        "Wohngebäude_Anteil":"buildings_anteil","Wohnungen_Anteil":"flats_anteil",
        "WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl":"flats"}, axis=1,inplace=True)

    # dont add gesamtenergie to plot
    viz = viz[viz["energy"] != "Wohngebäude Insgesamt"]
    viz = viz[viz["energy"] != "Wohnungen Insgesamt"]
    viz = viz[viz["energy"] != "Insgesamt"]
    
    obj = "Anzahl"

    if useWohnungen:
        column="flats"
        name = "Wohnungen in Wohngebäuden"
    else:
        column='buildings'
        name = "Wohngebäude"
    
    if asProportion:
        column += "_anteil"
        obj = "Anteile"

    #pivot data
    pivot = pd.pivot_table(viz, index='year',columns='energy', values=column)
    pivot = pivot.reset_index(level=0)

    #add bar per energy
    energies = viz["energy"].unique()
    data = []
    for energy in energies:
        data.append(go.Bar(name=energy, x=pivot["year"], y=pivot[energy]))

    #plot bar chart
    fig = go.Figure(data = data)
    fig.update_layout(barmode='stack',title= obj + " von Energiesystemen in fertiggestellten " + name +" in " + str(viz["Ort_Auspraegung_Label"].iloc[0]))
    fig.show()

    #plot line chart
    fig2 = go.Figure()
    colors = px.colors.qualitative.Plotly

    for i,energy in enumerate(energies):
        fig2.add_traces(go.Scatter(x=pivot['year'], y = pivot[energy], mode = 'lines', line=dict(color=colors[i%10]),name=energy))
    
    fig2.show()

def move_to_third_section(df,title,value):
    second_section = "2_" + title;
    third_section = "3_" + title;

    if "Merkmal" in title:
        df[third_section] = np.where((df[second_section] == value), df[second_section],df[third_section])
        #df[second_section] = np.where((df[second_section] == value),"WOHNGEBAUDE GESAMT",df[third_section])

    elif "Auspraegung" in title:
        merkmal = second_section.replace("Auspraegung","Merkmal")
        df[third_section] = np.where((df[merkmal] == value), df[second_section],df[third_section])
        #df[second_section] = np.where((df[merkmal] == value),"WOHNGEBAUDE GESAMT",df[third_section])

    else:
        print("unknown title: ", title)

    return df


def preprocess_dataset_kreise(df):
    

    #AGS Germany ist jetzt 0
    df["1_Auspraegung_Code"] = df["1_Auspraegung_Code"].replace({"DG":"0"})    
    df["1_Auspraegung_Code"] = pd.to_numeric(df["1_Auspraegung_Code"])

    #clean up multi level format of raw data 
    df["2_Auspraegung_Label"] = df["2_Auspraegung_Label"].replace({"Insgesamt":'Wohnungen Insgesamt'})  
    #df["2_Auspraegung_Label"] = df["2_Auspraegung_Label"].str.replace('Insgesamt','Wohnungen Insgesamt',regex=True)
    df.loc[df["2_Auspraegung_Code"] == "INSGESAMT","2_Auspraegung_Label"] = "Wohngebäude Insgesamt"

    titles = ["Merkmal_Code","Merkmal_Label","Auspraegung_Code","Auspraegung_Label"]

    for title in titles:
        if "Code" in title:
            df = move_to_third_section(df,title,"BAUPHE")
        elif "Label" in title:
            df = move_to_third_section(df,title,"Primär verwendete Heizenergie")

    for title in reversed(titles):
        title = "2_" + title
        df[title] = np.where((df["2_Merkmal_Code"] == "BAUPHE"),"Wohngebäude Insgesamt",df[title])

    #rename columns
    df.columns = df.columns.str.replace("1_", "Ort_")
    df.columns = df.columns.str.replace("2_", "Wohnort_")
    df.columns = df.columns.str.replace("3_", "Heizenergie_")


    #make building counts numeric
    building_counts = ["BAU017__Wohngebaeude_insgesamt__Anzahl","WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl","BAU018__Nichtwohngebaeude_insgesamt__Anzahl"]
    
    for building in building_counts:
        df[building] = df[building].str.replace('-','0',regex=True)
        df[building].fillna(0,inplace=True)
        df[building] = pd.to_numeric(df[building])

    print(df)

    #set total number of buildings for each year and place
    years = list(range(2016, 2021))
    orte = df["Ort_Auspraegung_Code"].unique()

    for ort in orte:
        for year in years:
            isLocation = (df["Ort_Auspraegung_Code"] == ort)
            isYear = (df["Zeit"] == year)

            gesamt = df.loc[isLocation & isYear & (df["Heizenergie_Auspraegung_Code"] == "INSGESAMT")].copy()
            
            gesamt_wohngebaeude = gesamt.loc[(gesamt["Wohnort_Auspraegung_Label"] == "Wohngebäude Insgesamt"),"BAU017__Wohngebaeude_insgesamt__Anzahl"]
            gesamt_wohnungen = gesamt.loc[(gesamt["Wohnort_Auspraegung_Label"] == "Wohnungen Insgesamt"),"WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl"]

            if(gesamt_wohngebaeude.size == 1 and gesamt_wohnungen.size == 1):
                df.loc[isLocation & isYear,"Wohngebäude_Gesamt"] = int(gesamt_wohngebaeude.item())
                df.loc[isLocation & isYear,"Wohnungen_Gesamt"] = int(gesamt_wohnungen.item())
            else:
                print("error: gesamt is wrong",gesamt)

    #calculate proportions

    df["Wohngebäude_Anteil"] = df["BAU017__Wohngebaeude_insgesamt__Anzahl"] / df["Wohngebäude_Gesamt"] * 100
    df["Wohnungen_Anteil"] = df["WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl"] / df["Wohnungen_Gesamt"] * 100


    return df




In [ ]:
#https://www.regionalstatistik.de/genesis/online?operation=previous&levelindex=3&step=2&titel=Tabellenaufbau&levelid=1655207886087&levelid=1655207857464#abreadcrumb
#Fertigstellungen neuer Wohngebäude und Nichtwohngebäude sowie Wohnungen in Wohngebäuden nach Zahl der Wohnungen und vorwiegender Art der Beheizung - Jahressumme

df_kreise_org = pd.read_csv("data/newbuildings_kreise.csv",delimiter=";", encoding='latin1')
df_kreise = df_kreise_org.copy()


df_kreise = preprocess_dataset_kreise(df_kreise)
df_kreise.to_csv("data/newbuildings_kreise_output.csv")


In [ ]:
####

# here, you can plot the data for each landkreis.
# parameters:
# viz : df dataframe slice of your data, for example a specific landkreis
# asProportion : bool, optional
#       if true, the visualisation will use proportion. otherwise, the total number is used. default is true
# useWohnungen : bool, optional
#       if true, flats will be counted. otherwise, buildings will be counted (which maybe have multiple flats in them). default is false

####

plot_heating_timeseries(df_kreise[df_kreise["Ort_Auspraegung_Code"] == 1001],False,True)

In [ ]:
#plot_heating_timeseries(df_kreise[df_kreise["Ort_Auspraegung_Code"] == 1001],False,True)
# Preparation of Workshop : extract data for "Mayen-Koblenz"

mk = df_kreise[df_kreise["Ort_Auspraegung_Code"] == 7137]
mk.to_csv("mayenkoblenz2.csv")

plot_heating_bars(mk,True,True)


mk = mk[mk["Wohnort_Auspraegung_Label"] == "Wohnungen Insgesamt"]
#pivot data
pivot = mk.pivot_table(index='energy',columns='year', values="flats_anteil")
pivot = pivot.reset_index(level=0)

pivot.to_csv("mayenkoblenz_resahpe.csv")
#df_kreise

In [ ]:
# Flensburg Data as test
sh = df_kreise.copy()
sh = sh[sh["Ort_Auspraegung_Code"] == 11005005]
sh.rename({"Zeit": "year","BAU017__Wohngebaeude_insgesamt__Anzahl":"buildings","Heizenergie_Auspraegung_Label":"energy"}, axis=1,inplace=True)

plot_heating_timeseries(sh)


In [ ]:
##choose on of those labels
#print("Auswahl an Wohngebäuden:")
#print(df2_org["2_Auspraegung_Label"].unique())

#dataset all of germany
df2_org = pd.read_csv("data/newbuildings_germany.csv",delimiter=";", encoding='latin1')
df2 = df2_org.copy();

df2_viz = preprocess_dataset_germany_by_building_type(df2,"Wohngebäude")

#ploz
plot_heating_timeseries(df2_viz)
